## Identify Fraud from Enron Email

### Overview

In 2000, Enron was one of the largest companies in the United States. By 2002, it had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, a significant amount of typically confidential information entered into the public record, including tens of thousands of emails and detailed financial data for top executives.

The purpose of this project is to use machine learning skills to identify Enron Employees who may have committed fraud, based on the public Enron financial and email dataset. I have performed an end-to-end process of investigating the data through a machine learning lens.

### Dataset

The data is combined with a hand-generated list of persons of interest in the fraud case, which means individuals who were indicted, reached a settlement or plea deal with the government, or testified in exchange for prosecution immunity.